<a href="https://colab.research.google.com/github/HassanRiaz1992/Resume-Parser-in-Spacy-3/blob/main/Resume_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


In [3]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [4]:
!pip install pyresparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 112.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 98.8 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3957 sha256=095f2a487612018bdc45cefc75fd48b9cf3a7527c430ceb0b614d99a2c0b2b65
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import pandas as pd

In [6]:
#cv_data = pd.read_pickle(r'/content/sample_data/Resume/train_data.pkl')

In [7]:
cv_data = json.load(open('/content/train_data.json'))

In [8]:
len(cv_data)

200

In [20]:
!python -m spacy init fill-config /content/base_config.cfg config.cfg

2023-06-15 17:32:49.641821: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [21]:
cv_data[3]

["Ijas Nizamuddin Associate Consultant - State Street  Irinchayam B.O, Kerala - Email me on Indeed: indeed.com/r/Ijas- Nizamuddin/6748d77f76f94eed  With close to 3 years of experience in IT industry, I have had excellent exposure to design, development and implementation of Client Server Applications in various domains such as Banking and Finance concepts. I have been involved in various software Development projects in Open System environment.  WORK EXPERIENCE  Associate Consultant  Oracle Corporation -  June 2011 to Present  State Street Global Advisors (SSgA) is the asset management business of State Street Corporation, one of the world's leading providers of financial services to institutional investors1, with a heritage dating back over two centuries. Backed by the strength and stability of the State Street organization, SSgA makes continual investments in asset management and client service platform, resulting in a client-focused, solutions-driven orientation .BrokerViews is the 

In [22]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en') #created an empty pipeline for english language
    db = DocBin() # created a doc bin object
      # iterate the dataset which we want to convert into spacy format
    for text, annot in tqdm(data):
        # convert the dataset of resume into text format
        doc = nlp.make_doc(text)
        # identify the annotation from data
        annot = annot['entities']
        # store the entities into an array
        ents = []
        # Number of characteristics
        entity_indices = []

        # Extracting information of entities from the data set in iteration
        #start, end and label are like 16, 36, 'Designation'
        for start, end, label in annot:
            # No Entity shall be skip from extraction
            skip_entity = False
            # if an entity character is already present in the entity_indices than we shall skip the entity and and cotinure the process
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue
            # entity indices are passed to the array of entity_indices and list of star and end maintained
            entity_indices = entity_indices + list(range(start, end))
             # we need to check that either annotated data is producing error or not
            try:
                span = doc.char_span(start, end,label=label,alignment_mode = 'strict')
            except:
                continue
              # if span has nothing than we need tocheck that at span has error
            if span is None:
                err_data = str([start, end]) + " " + str(text) + "\n "
                file.write(err_data)
            else:
                ents.append(span)
        # putting all the extracted entities in the in doc which later added to docbin
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass
    return db

In [23]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.1)

In [24]:
 len(train),len(test)

(180, 20)

In [25]:
file = open('error.txt', 'w', encoding='utf-8')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 20/20 [00:00<00:00, 44.77it/s]


In [26]:

len(db.tokens)

20

In [16]:
#import subprocess
# Spacy command
#config_path = "/content/config.cfg"
#train_data_path = "./content/train_data.spacy"
#test_data_path = "./content/test_data.spacy"
#output_path = "./output"

#command = ["python", "-m", "spacy", "train", config_path, "--output", output_path, "--paths.train", train_data_path, "--paths.dev", test_data_path]
#subprocess.run(command)

In [27]:
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-06-15 17:34:03.943974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-15 17:34:16,472] [INFO] Set up nlp object from config
[2023-06-15 17:34:16,488] [INFO] Pipeline: ['transformer', 'ner']
[2023-06-15 17:34:16,491] [INFO] Created vocabulary
[2023-06-15 17:34:16,491] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT 

In [32]:
NLP = spacy.load('/content/output/model-best')

In [43]:
import PyPDF2

# Function to extract text from PDF
def pdf_to_text(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        for page in range(num_pages):
            pdf_page = pdf_reader.pages[page]
            text += pdf_page.extract_text()

    return text

# Provide the path to your PDF file
pdf_path = '/content/Smith_Resume.pdf'

# Convert PDF to text
cv_text = pdf_to_text(pdf_path)

# Print the extracted text
print(cv_text)


Michael Smith 
BI / Big Data/ Azure  
Manchester , UK- Email me on Indeed:  indeed.com/r/ falicent/140749dace5dc26f  
 
10+ years of Experience in Designing, Development, Administration, Analysis, 
Management inthe Business Intelligence Da ta warehousing, Client Server 
Technologies, Web -based Applications, cloud solutions and Databases.  
Data warehouse: Data analysis, star/ snow flake schema data modeling and design 
specific todata warehousing and business intelligence environment.  
Database: Experience in database designing, scalability, back -up and recovery, 
writing andoptimizing SQL code and Stored Procedures, creating functions, views, 
triggers and indexes.  
Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL 
Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure 
data lake analytics(U -SQL). 
Big Data: Worked Azure data lake store/analytics for big data processing and Azure 
data factoryto schedule U -SQL jobs. Designed 

In [44]:
doc = NLP(cv_text)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

NAME   =>   Michael Smith
LOCATION   =>   Manchester
EMAIL ADDRESS   =>   indeed.com/r/ falicent/140749dace5dc26f
COMPANIES WORKED AT   =>   Microsoft
DESIGNATION   =>   Software Engineer
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COMPANIES WORKED AT   =>   Microsoft
COLLEGE NAME   =>   The University of Manchester
GRADUATION YEAR   =>   2007
SKILLS   =>   problem solving (Less than 1 year), project lifecycle (Less than 1 year), project 
manager (Less than 1 year), technical assistance. (Less than 1 year)


In [45]:
# Convert PDF to text
cv_text = pdf_to_text('/content/Resume Hassan (1).pdf')

# Print the extracted text
print(cv_text)


 Hassan Riaz 
Contact  
Address:  
Flat 07 , Block C-6, ZTBL Colony , G-
7/2, Islamabad  
 
Phone:  
+92 0331 5547995   
 
Email:  
tipus816@gmail.com  
 
Languages  
Urdu  – A1 
English  – A2 
Hobbies  
 Reading  
 Writing  
 
Repository  
https://github.com/HassanRiaz19
92 Summary  
Seeking a job in challenging and healthy working environment 
where I cherish challenges and aspire to seek excellence in the task 
assigned to me on the basis of my knowledge and experience . 
Skill Highlights  
 Mult i tasking  
 Strong decision maker  
 Complex problem 
solver   Creative  
 Innovative  
 
Experience  
Officer Grade II  - 07/201 7 to Till date  
Zarai Taraqiati Bank Limited , Islamabad  
 Preparation evaluation and submission of Sales Tax 
Return along with reconciliation of Other Indirect Taxes . 
 Evaluation for medical expense and it budgeting.  
 
Statistical Assistant  - 02/201 7 to 07/2017  
Pakistan Bureau of Statisitcs , Islamabad  
 
 Managing Census Operation of FATA

In [46]:
doc = NLP(cv_text)
for ent in doc.ents:
    print(ent.label_.upper(),"  =>  ", ent.text)

NAME   =>   Hassan Riaz
DEGREE   =>   Masters  of Science: Computer Science
GRADUATION YEAR   =>   2023
DEGREE   =>   Masters of Science: Economics
GRADUATION YEAR   =>   2015
